## Main VTK code

In [1]:
from trame.app import get_server, jupyter
from trame.widgets import vtk, trame, vuetify, html
from trame.ui.vuetify import SinglePageLayout, VAppLayout

from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkRenderer,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkPolyDataMapper,
    vtkActor,
)
import vtkmodules.vtkRenderingOpenGL2  # noqa
from vtkmodules.vtkInteractionStyle import vtkInteractorStyleSwitch  # noqa

# VTK -----------------------------------------------------

DEFAULT_RESOLUTION = 6

renderer = vtkRenderer()
renderWindow = vtkRenderWindow()
renderWindow.AddRenderer(renderer)

renderWindowInteractor = vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.GetInteractorStyle().SetCurrentStyleToTrackballCamera()

cone_source = vtkConeSource()
mapper = vtkPolyDataMapper()
actor = vtkActor()
mapper.SetInputConnection(cone_source.GetOutputPort())
actor.SetMapper(mapper)
renderer.AddActor(actor)
renderer.ResetCamera()
renderWindow.Render()

# Trame ---------------------------------------------------

server = get_server()
state, ctrl = server.state, server.controller


def reset_resolution():
    state.resolution = 6


@state.change("resolution")
def update_resolution(resolution, **kwargs):
    cone_source.SetResolution(int(resolution))
    ctrl.view_update()
    


ctrl.on_server_ready.add(lambda **_: print("Ready"))
ctrl.on_server_exited.add(lambda **_: print("Exited"))
ctrl.on_server_ready.add(ctrl.view_update)


## Main View

In [2]:
main_layout = SinglePageLayout(server)

with state:
    with main_layout:
        # Validate client life cycle
        trame.LifeCycleMonitor(events=("['created']",))

        main_layout.icon.click = ctrl.reset_camera
        main_layout.title.set_text("Cone")
        main_layout.toolbar.dense = True

        # Toolbar
        with main_layout.toolbar as toolbar:
            vuetify.VSpacer()
            vuetify.VSlider(
                # label="Hello2",
                hide_details=True,
                v_model=("resolution", 6),
                max=60,
                min=3,
                step=1,
                style="max-width: 300px;",
            )
            vuetify.VSwitch(
                hide_details=True,
                v_model=("$vuetify.theme.dark",),
            )
            with vuetify.VBtn(icon=True, click=reset_resolution):
                vuetify.VIcon("mdi-undo")

        with main_layout.content:
            with vuetify.VContainer(fluid=True, classes="pa-0 fill-height"):
                view = vtk.VtkRemoteView(renderWindow, ref="remoteView")
                ctrl.view_update.add(view.update)
                ctrl.reset_camera.add(view.reset_camera)

## Local View

In [3]:
local_layout = VAppLayout(server, "local")

with local_layout:
    with vuetify.VContainer(fluid=True, classes="pa-0 fill-height"):
        view = vtk.VtkLocalView(renderWindow, ref="localView")
        ctrl.view_update.add(view.update)
        ctrl.reset_camera.add(view.reset_camera)

## Client View

In [4]:
client_layout = VAppLayout(server, "client")

with client_layout:
    with vuetify.VContainer(fluid=True, classes="pa-0 fill-height"):
        with vtk.VtkView(ref="clientView") as view:
            ctrl.reset_camera.add(view.reset_camera)
            with vtk.VtkGeometryRepresentation():
                vtk.VtkAlgorithm(
                    vtkClass="vtkConeSource", state=("{ resolution }",)
                )

## Output

In [5]:
jupyter.show(server, "local", height=200)
jupyter.show(server, "client", height=200)
jupyter.show(server, height=200)

Ready


In [6]:
server.running

True

In [7]:
await server.stop()

Exited


In [8]:
server.running

False